In [1]:
!pip install -r requirements.txt 

  Using cached pandas_profiling-3.2.0-py2.py3-none-any.whl.metadata (21 kB)
INFO: pip is looking at multiple versions of pandas-profiling to determine which version is compatible with other requirements. This could take a while.
  Using cached pandas_profiling-3.1.0-py2.py3-none-any.whl.metadata (23 kB)
  Using cached pandas_profiling-3.0.0-py2.py3-none-any.whl.metadata (21 kB)
  Using cached pydantic-2.12.5-py3-none-any.whl.metadata (90 kB)
  Using cached visions-0.7.1-py3-none-any.whl.metadata (5.5 kB)
  Using cached htmlmin-0.1.12.tar.gz (19 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [27 lines of output]
      Traceback (most recent call last):
        File "/home/leo/Documents/dev/oca-ia/.venv/lib/python3.13/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 389, in <module>
     

In [7]:
import pandas as pd
from PIL import Image, ImageChops
import numpy as np
import torch
from lib.ImageFIlter import treat_image_PIL
from sklearn.model_selection import KFold
from torch.utils.data import  DataLoader, TensorDataset, Dataset
from torch import nn
import timm
from tqdm.notebook import tqdm
import torch.optim as optim
import os

## Exemplo de classificador com leitura do CSV. 


In [8]:
## Criacao de pastas output e models se nao existirem
import os
if not os.path.exists('output'):
    os.makedirs('output')
if not os.path.exists('models'):
    os.makedirs('models')

In [9]:
ds_file = pd.read_csv('dataset/oca.csv')
ds_file = ds_file[['path','oca']]
ds_file.rename(columns={'oca':'label'}, inplace=True)

In [10]:
ds_file.to_csv('dataset/oca_incor.csv', index=False)

## Geracao Tensor com imagens filtradas

In [11]:
## Loop
img_dataset = np.ones((ds_file.shape[0],3,256,256),dtype=np.uint8)

j=0
for i in ds_file['path']:
    img_dataset[j]=treat_image_PIL('dataset/path/'+i,2)
    j+=1
tensor_imagem = torch.tensor(img_dataset)



## Geracao de rótulos para classificação

Colunas utilizadas para classificacao (Labels)

labels = [ OCA, NOCA]

In [13]:
##
ds_file.columns
labels = [ 'label']
ds_labels = ds_file[labels]
ds_labels=ds_labels.astype(int)#,'False':0})
tensor_label = torch.tensor(np.array(ds_labels))
tensor_label

tensor([[0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [1],
        [0],
        [1],

### Resumo:

model(tensor_imagem,ds_id_sex_et)

labels : tensor_label

## Criacao Subset 



In [14]:
class Subset(Dataset):
    r"""
    Subset of a dataset at specified indices.

    Arguments:
        dataset (Dataset): The whole Dataset
        indices (sequence): Indices in the whole set selected for subset
    """
    def __init__(self, dataset, indices):
        self.dataset = dataset
        self.indices = indices

    def __getitem__(self, idx):
        return self.dataset[self.indices[idx]]

    def __len__(self):
        return len(self.indices)

    @property
    def classes(self):
        return self.data.classes

    def shape(self):
        return self.dataset

## K-fold = 10


In [15]:
kf = KFold(n_splits=5)
kf.get_n_splits(tensor_imagem)
print(kf)
#for i in enumerate(kf.split(tensor_imagem)):
#    print(i)

KFold(n_splits=5, random_state=None, shuffle=False)


## Criacao do dataset de treino e validacao com batchsize

In [16]:
BATCH_SIZE = 16
train_dataset = TensorDataset(tensor_imagem, tensor_label)
train_loader_img = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [12]:
train_dataset[0][0]

tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]], dtype=torch.uint8)

## Create a model to concat layers

O modelo utilizado será o mesmo do artigo: Artificial Intelligence-Driven Screening System for Rapid Image-Based Classification of 12-Lead ECG Exams: A Promising Solution for Emergency Room Prioritization 

Desenvolvido conforme a figura abaixo: 

![image.png](attachment:image.png)

In [17]:
#from models import ECGClassifierResnet

class ECGClassifierResnet(nn.Module):
    def __init__(self, num_classes=5):
        super(ECGClassifierResnet, self).__init__()
        # Where we define all the parts of the model
        #self.base_model = timm.create_model('efficientnet_b0', pretrained=True) 
        self.base_model=timm.create_model('resnet50d.ra4_e3600_r224_in1k',pretrained=True)
        #self.base_model = timm.create_model('vit_mediumd_patch16_reg4_gap_256.sbb2_e200_in12k_ft_in1k',num_classes=5,pretrained=True)


        self.features = nn.Sequential(*list(self.base_model.children())[:-1])

        enet_out_size = 2048        # Make a classifier
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(enet_out_size,1)
        ) # saida como Sigmoid multilabel

    def forward(self, x):
        # Connect these parts and return the output
        x = self.features(x)
        output = self.classifier(x)
        #output = nn.Softmax(dim=1)(x)
        return output

## Train - Valid Loop

Fluxo de treinamento

Input: modelo, dataloader_train, dataloader_teste , epocas.
- Para cada epoca

    - treinamento:

        - para cada batch (lote de imagens) no dataloader_train: (executo para todo o dataloader)
            
            leitura das imagens e rotulos. 
            
            prediz o rotulo (outputs): rotulos preditos para o tamanho do batch (ex: para um batch de 10, o 
            output tem tamanho 10)
            
            calcula o Loss () 
            
            aplica o backward -> Ajuste dos pesos nos neuronios de acordo com o valor de loss (w_{i})

            otimizo o modelo com optimizer.step()

    - validacao (avaliacao do desempenho para aquele conjunto de treinamento):
        - para cada batch (lote de imagens) no dataloader_train: (executo para todo o dataloader)

            leitura das imagens e rotulos.

            prediz o rotulo (outputs): rotulos preditos para o tamanho do batch (ex: para um batch de 10, o 
            output tem tamanho 10)
            
            calcula o Loss ()


    - Calculo de metricas de loss media de validacao.

    - Calculo de metrica de loss media de treinamento. 

    - Print de metricas de desempenho para a epoca. 








In [19]:
def salvar_model(model, path = 'output/modelos', name_file='model.pth'):
    full_path = os.path.join(path, name_file)
    torch.save(model.state_dict(), full_path)

In [20]:
def salvar_metricas(path='output/metricas', name_file_train='train_loss_total.npy', name_file_val='val_loss_total.npy',predict_label=None, true_label=None):
    full_path_train = os.path.join(path, name_file_train)
    full_path_val = os.path.join(path, name_file_val)
    np.save(full_path_train, np.array(predict_label))
    np.save(full_path_val, np.array(true_label))

In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: " + str(device))
def simple_loop(model, train_image, val_image, epochs):
    # Simple training loop
    num_epochs = epochs
    train_losses, val_losses = [], []
    lim_loss = 1.5
    iter_size = 5
    print(f'Number of training images per iteration: {iter_size}')
    #model = modelo( num_classes=5)
    model.to(device)
    criterion =  nn.L1Loss()
    #criterion =  nn.CrossEntropyLoss()
    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    predict_label_full_epoch = []
    predict_label_full = []
    true_label_full_epoch = []
    true_label_full = []
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss_train = 0.0
        for images, labels in tqdm(train_image, desc='Training loop'):
            # Move inputs and labels to the device
            images = images.to(torch.float)
            image, label = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(image)
            #print(outputs)
            #print(label)
            loss_train = criterion(outputs.float(), label.float())
            loss_train.backward()
            optimizer.step()
            running_loss_train += loss_train.item() * label.size(0)
        train_loss = running_loss_train / len(train_image.dataset)
        train_losses.append(train_loss)
        model.eval()
        running_loss_valid = 0.0
        rotulos =[] 
        predict_label =[]
        true_label =[]
        _iter=0
        with torch.no_grad():
            for images, labels in tqdm(val_image, desc='Validation loop'):
                # Move inputs and labels to the device
                images = images.to(torch.float)
                images, label = images.to(device), labels.to(device)
                rotulos.append(label.cpu().data.numpy())
                outputs = model(images)
                loss_valid = criterion(outputs.float(), label.float())
                #print( [outputs.cpu().data.numpy().astype(int).T[0]])
                #print(label.cpu().data.numpy().astype(int).T[0])
                #print(predict_label)
                try:
                    _pred = outputs.cpu().data.numpy().astype(int).T[0].tolist()
                    if(len(_pred) == iter_size):
                        predict_label.append(_pred)
                        _true = label.cpu().data.numpy().astype(int).T[0].tolist()
                        true_label.append(_true)
                except Exception as e:
                    print(f"Concatenation error iter: {e}")
                    print(_pred)
                    print(predict_label)    
                running_loss_valid += loss_valid.item() * label.size(0)
                _iter +=1
        val_loss = running_loss_valid / len(val_image.dataset)
        val_losses.append(val_loss)
        print(f'End validation for epoch {epoch}')
        print(f'Amount of images validated: {val_image}')
        print(f'Label predict shape : {len(predict_label)} for epoch {epoch}')
        print(f'Count of iterations: {_iter} for epoch {epoch}')
        try:
            _p = predict_label
            _t = true_label
            predict_label_full.append(_p)
            true_label_full.append(_t)
        except Exception as e:
            print(f"Concatenation error full: {e}")
            print(predict_label)
            print(true_label)
        #val_acc = accuracy_score(rotulos,output_model)
        print(f'Val accuracy {epoch}:')
        print(f"Epoch {epoch+1}/{num_epochs} - Train loss: {train_loss}, Validation loss: {val_loss}")

    predict_label_full_out = np.array(predict_label_full)
    true_label_full_out = np.array(true_label_full)
    salvar_metricas(path='output/metricas', name_file_train=f'predict_label_{i}.npy', name_file_val=f'true_label_{i}.npy', predict_label= outputs, true_label= labels)

    return train_losses, val_losses, model,predict_label_full_out, true_label_full_out

Device: cuda:0


In [ ]:

train_loss_total = []
val_loss_total =[]
all_models =[]
epochs = 5
## create first model.
for i, (train_index, test_index) in enumerate(kf.split(train_dataset)):
    print(f"Fold {i}:")
    #print(f"  Train: index={train_index}")
    #print(f"  Test:  index={test_index}")
    ## init train test for folder
    train_dataset_part = Subset( train_dataset, train_index)
    val_dataset_part = Subset( train_dataset, test_index)

    train_loader_img = DataLoader(train_dataset_part, batch_size=5, shuffle=True)
    val_loader_img = DataLoader(val_dataset_part, batch_size=5, shuffle=True)

    model= ECGClassifierResnet( num_classes=1)
    salvar_model(model, path='output/modelos', name_file=f'model_fold_{i}.pth')
    print(f'Train and valid for Fold {i}')
    t, l,_,outputs,labels = simple_loop(model, train_loader_img,val_loader_img, epochs)
    ## Evaluate model.
    train_loss_total.append(t)
    val_loss_total.append(l)

Fold 0:
Train and valid for Fold 0
Number of training images per iteration: 5


Training loop:   0%|          | 0/35 [00:00<?, ?it/s]

Validation loop:   0%|          | 0/9 [00:00<?, ?it/s]

End validation for epoch 0
Amount of images validated: <torch.utils.data.dataloader.DataLoader object at 0x7c7b302779d0>
Label predict shape : 8 for epoch 0
Count of iterations: 9 for epoch 0
Val accuracy 0:
Epoch 1/5 - Train loss: 0.6547354110624901, Validation loss: 5882.8451149680395


Training loop:   0%|          | 0/35 [00:00<?, ?it/s]

...

Validation loop:   0%|          | 0/9 [00:00<?, ?it/s]

End validation for epoch 3
Amount of images validated: <torch.utils.data.dataloader.DataLoader object at 0x7c7b30287050>
Label predict shape : 8 for epoch 3
Count of iterations: 9 for epoch 3
Val accuracy 3:
Epoch 4/5 - Train loss: 0.5057160468459818, Validation loss: 3.476217269897461


Training loop:   0%|          | 0/35 [00:00<?, ?it/s]

Validation loop:   0%|          | 0/9 [00:00<?, ?it/s]

End validation for epoch 4
Amount of images validated: <torch.utils.data.dataloader.DataLoader object at 0x7c7b30287050>
Label predict shape : 8 for epoch 4
Count of iterations: 9 for epoch 4
Val accuracy 4:
Epoch 5/5 - Train loss: 0.5069192563522757, Validation loss: 0.4343366946938426


In [87]:
outputs
## formato ddos outputs [epoch][batch][labels]
#out = np.array(outputs)

array([[[31616, 31616, 31616, 31616, 31616],
        [31616, 31616, 31616, 31616, 31616],
        [31616, 31616, 31616, 31616, 31616],
        [31616, 31616, 31616, 31616, 31616],
        [31616, 31616, 31616, 31616, 31616],
        [31616, 31616, 31616, 31616, 31616],
        [31616, 31616, 31616, 31616, 31616],
        [31616, 31616, 31616, 31616, 31616]],

       [[   17,    17,    17,    17,    17],
        [   17,    17,    17,    17,    17],
        [   17,    17,    17,    17,    17],
        [   17,    17,    17,    17,    17],
        [   17,    17,    17,    17,    17],
        [   17,    17,    17,    17,    17],
        [   17,    17,    17,    17,    17],
        [   17,    17,    17,    17,    17]],

       [[    2,     2,     2,     2,     2],
        [    2,     2,     2,     2,     2],
        [    2,     2,     2,     2,     2],
        [    2,     2,     2,     2,     2],
        [    2,     2,     2,     2,     2],
        [    2,     2,     2,     2,     2],
      

In [ ]:
np.array(labels[0][0:-2])

array([[1, 1, 0, 0, 0],
       [0, 1, 1, 1, 0],
       [1, 0, 0, 1, 1],
       [0, 0, 1, 1, 1],
       [0, 0, 1, 1, 1],
       [1, 1, 0, 0, 0],
       [0, 0, 1, 0, 1]])

## Métricas de Avaliacao dos modelos

Acuracia

Precisao

Sensibilidade

Especificidade

F1-Score

In [88]:
#### Leitura de métricas salvas
import numpy as np
predict_label_0 = np.load('output/metricas/predict_label_0.npy')
true_label_0 = np.load('output/metricas/true_label_0.npy')
# --- IGNORE ---


array([[[5090, 4552, 5090, 5090, 5090],
        [9356, 5090, 3771, 5021, 5090],
        [5090, 5090, 3520, 5090, 5453],
        [5090, 5090, 5090, 9419, 5090],
        [5090, 5090, 5090, 5090, 4520],
        [5090, 5090, 4277, 5090, 5090],
        [3335, 5090, 7920, 5090, 5090],
        [5090, 5090, 5017, 5281, 5090]],

       [[   0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0]],

       [[  31,   31,    1,    3,   31],
        [  31,   31,   31,    1,   31],
        [  31,   31,    4,   31,   31],
        [  31,   31,    2,   31,   31],
        [  20,   31,   31,    0,   31],
        [  77,   11,   31,   31,   31],
        [  31,    4,   31,    1,   31],
        [   2,   31,   39,    9,   31]],

       [[   0,    1,    1,    1,  

In [ ]:
220/16

13.75

In [ ]:
print("a")
print(type(predict_label_0))
predict_label_0.shape

a
<class 'numpy.ndarray'>


(102, 5)

## Criacao graficos de treinamento e validacao

In [ ]:
## Criar graficos de treinamento e validacao
import matplotlib.pyplot as plt
for i in range(5):
    plt.figure(figsize=(10,5))
    plt.plot(train_loss_total[i], label='Train Loss')
    plt.plot(val_loss_total[i], label='Validation Loss')
    plt.title(f'Fold {i} - Train and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(f'output/graficos/loss_fold_{i}.png')
    plt.show()
    plt.close()
